In [ ]:
import pandas as pd
import boto3
import zipfile
import os
from tqdm import tqdm
import shutil

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Setting up for AWS

In [ ]:
destination = '/content/gdrive/My Drive/safegraph-sdm' #enter your desired Google Drive folder without a '/' at the end

accessKey = 'RDT348CQNBUUIXW8ITTT'
secretKey = 'vYeorvQZdFnS6FM2ADXN8tNzzbydEfuJcXZfbFp3'

session = boto3.Session(
    aws_access_key_id=accessKey, # Found at: https://catalog.safegraph.io/app/browse -- under start here and 'Reveal Access Key'
    aws_secret_access_key=secretKey, # Found at: https://catalog.safegraph.io/app/browse -- under start here and 'Reveal Access Key'
    region_name = 'us-east-1' # Note this used to be us-west-2 --- it has changed to us-east-1
)

bucket = 'sg-c19-response'
prefix = 'social-distancing/v2'
s3 = session.client('s3', endpoint_url = 'https://s3.wasabisys.com')

# Functions

In [ ]:
def getCorrespondingDate(key):
  if 'csv' in key:
    if 'backfill' in key:
      date = '-'.join(key.split("/")[6:8])
      return(date)
    else:
      split = key.split("/")[2:4]
      date = '-'.join([split[0], str(int(split[1])).zfill(2)])
      return(date)
  return("")

def addEntry(date_dict, key):
  date = getCorrespondingDate(x['Key'])
  if date != "":
    if date in date_dict.keys():
      date_dict[date] += [key]
    else:
      date_dict[date] = [key]
  return(date_dict)

def readFilterSaveDel(file, outfile):
  df = pd.read_csv(file)
  df['origin_census_block_group'] = df['origin_census_block_group'].astype(str).str.zfill(12)
  df1 = df[df.origin_census_block_group.str.startswith('12')]
  df1.to_csv(outfile)
  deleteFile(file)

def deleteFile(file):
  open(file, 'w').close() #rewrite the file to contain zero data to save storage
  os.remove(file) #send file to trash, user can delete it from there if they want but it takes up no storage
  return(True)

## Date Dictionary

In [ ]:
#lists all objects in the S3 bucket and stores the paths by date in the date_dict
d = s3.list_objects(Bucket=bucket, Prefix=prefix)
date_dict = {}
for x in d['Contents']:
  date_dict = addEntry(date_dict, x['Key'])
date_dict

{'2019-01': ['social-distancing/v2/2019/01/01/2019-01-01-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/02/2019-01-02-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/03/2019-01-03-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/04/2019-01-04-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/05/2019-01-05-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/06/2019-01-06-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/07/2019-01-07-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/08/2019-01-08-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/09/2019-01-09-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/10/2019-01-10-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/11/2019-01-11-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/12/2019-01-12-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/13/2019-01-13-social-distancing.csv.gz',
  'social-distancing/v2/2019/01/14/2019-

# Cycle through each month, Download Filter and Save

In [ ]:
months = ['2020-07', '2020-08']
tag = '-social-distancing.csv.gz'

In [ ]:
for m in tqdm(months):
  for f in tqdm(date_dict[m]):
    dt = f.split('/')[-1][:-len(tag)]
    infile = '/'.join([destination, 'tmp', f.split('/')[-1]])
    outfile1 = 'fl-social-distancing-' + dt + '.csv'
    if outfile1 not in os.listdir(destination):
      s3.download_file(bucket, f, infile)
      readFilterSaveDel(infile, destination + '/' + outfile1)


100%|██████████| 2/2 [01:16<00:00, 38.42s/it]
